# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir('../')

# IMPORTS

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime, timedelta
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import json
import yt_dlp
import time
import os

path_read1          = "repository\\querys\\operation\\read\\game_event.sql"
path_read2          = "repository\\querys\\operation\\read\\game_event_filter.sql"
path_update         = "repository\\querys\\operation\\insert\\game_event.sql"

def ler_json_em_stream(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        for linha in f:
            try:
                yield json.loads(linha)
            except json.JSONDecodeError:
                continue
    return None

def wait(seconds=60):
    for _ in range(seconds):
        time.sleep(1)

# lendo tabela de eventos 
with open(path_read1, "r", encoding="utf-8") as file:
    query_read1     = file.read()

# lendo tabela de eventos 
with open(path_read2, "r", encoding="utf-8") as file:
    query_read2     = file.read()

# lendo tabela de eventos 
with open(path_update, "r", encoding="utf-8") as file:
    query_update1   = file.read()

# ESCREVENTOS OS EVENTOS NO BANCO DE DADOS

In [ ]:
df_events           = db_manager.read_df(query_read1)

# Exemplo de uso:
events = []
for item in ler_json_em_stream('data\\temp\\saida.json'):
    if item['type'] not in events:
        
        sql_template = Template(query_update1)
        rendered_sql = sql_template.render(
            ID_GAME = 1, # ID DO DOTA 2
            NAME    = item['type']
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error: {e}")
            db_manager.connection.rollback()
            
        events.append(item['type'])

# PARSEANDO OS DADOS E RECORTANDO OS EVENTOS

In [136]:
df_events_filter    = db_manager.read_df(query_read2)

PLAYER_SLOT = 7

"""
SO PEGA EVENTOS DO PLAYER, MAS TEM EVENTOS COMO TORRE, CURRIER QUE NÃO E DO PLAYER
"""

# Exemplo de uso:
structured_data     = {
    'time_Start': None,
    'steamids': {},
    'player_slot': {},
    'player_hero': {},
    'records': {}
}
events = df_events_filter["name"].values.tolist()
for item in ler_json_em_stream('data\\temp\\saida.json'):
    if item['type'] == 'draft_start':
        structured_data['time_Start'] = item['time']
        continue
    if structured_data['time_Start']: # SE JOGO TIVER TEMPO DIFERENTE DE NONE O GAME FOI INICIADO
        if item['type'] == 'steamid':
            structured_data['steamids'][item['key']] = item['unit']
            continue
        elif item['type'] == 'player_slot':
            structured_data['player_slot'][item['value']] = item['key']
            continue
        elif item['type'] == 'interval':
            if 'hero_id' in item and 'slot' in item and 'unit' in item:
                structured_data['player_hero'][item['slot']] = item['unit'].replace("CDOTA_Unit_Hero_", "").replace("npc_dota_hero_", "").lower()
                continue
        elif item['type'] in events:
            if PLAYER_SLOT in structured_data['player_hero']:
                if structured_data['player_hero'][PLAYER_SLOT] in (json.dumps(item, ensure_ascii=False)).lower():
                    if item['time'] not in structured_data["records"]:
                        structured_data["records"][item['time']] = []
                    time = item['time']
                    item = {key:value for key, value in item.items() if key != 'time'}
                    structured_data["records"][time].append(item)
structured_data['player_hero'][PLAYER_SLOT]

'rattletrap'

# CRIANDO OS RECORTES

In [140]:
from sklearn.cluster import DBSCAN
import numpy as np

times = pd.DataFrame({"times": sorted(list(structured_data["records"].keys()))})

if not times.empty:

    # DBSCAN espera dados em formato 2D
    X = times['times'].values.reshape(-1, 1)

    # eps=10 para diferença máxima de 10s, min_samples=1 para considerar todos os pontos
    dbscan = DBSCAN(eps=10, min_samples=1, metric='euclidean')
    labels = dbscan.fit_predict(X)

    # Adiciona os rótulos dos clusters ao DataFrame
    times['cluster'] = labels

    # Para cada cluster, encontre o tempo mínimo e máximo
    cluster_ranges = (
        times.groupby('cluster')['times']
        .agg(['min', 'max'])
        .reset_index()
        .rename(columns={'min': 'time_min', 'max': 'time_max'})
    )

    cluster_ranges["time_record"] = cluster_ranges["time_max"] - cluster_ranges["time_min"]
    for idx, row in cluster_ranges[cluster_ranges["time_record"] < 10].iterrows():
        cluster_ranges.at[idx, "time_min"] = float(row['time_min'] - (10 - row['time_record'])/2)
        cluster_ranges.at[idx, "time_max"] = float(row['time_max'] + (10 - row['time_record'])/2)
    cluster_ranges["time_record"] = cluster_ranges["time_max"] - cluster_ranges["time_min"]

    # Unir intervalos sobrepostos e corrigir tempos mínimos e máximos
    def merge_overlapping_intervals(df):
        # Ordena pelo tempo mínimo
        intervals = df.sort_values('time_min').to_dict('records')
        merged = []
        for interval in intervals:
            if not merged:
                merged.append(interval)
            else:
                last = merged[-1]
                # Se houver sobreposição
                if interval['time_min'] <= last['time_max']:
                    # Atualiza o tempo máximo do último intervalo
                    last['time_max'] = max(last['time_max'], interval['time_max'])
                    last['time_min'] = min(last['time_min'], interval['time_min'])
                    last['time_record'] = last['time_max'] - last['time_min']
                    # Opcional: pode manter uma lista de clusters unidos se quiser
                else:
                    merged.append(interval)
        return pd.DataFrame(merged)

    cluster_ranges = merge_overlapping_intervals(cluster_ranges)

    total_seconds = int(cluster_ranges['time_record'].sum())
    print(str(timedelta(seconds=total_seconds)))

0:06:30


C:\Users\johnfreitas\AppData\Local\Temp\ipykernel_3748\236918563.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '63.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cluster_ranges.at[idx, "time_min"] = float(row['time_min'] - (10 - row['time_record'])/2)
C:\Users\johnfreitas\AppData\Local\Temp\ipykernel_3748\236918563.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '73.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  cluster_ranges.at[idx, "time_max"] = float(row['time_max'] + (10 - row['time_record'])/2)
